In [0]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

import tensorflow as tf
from tensorflow.python.eager import context
from tensorflow.python.framework import ops
from tensorflow.python.ops import control_flow_ops
from tensorflow.python.training import optimizer


class Lookahead(optimizer.Optimizer):
    '''Tensorflow implementation of the lookahead wrapper.
    Lookahead Optimizer: https://arxiv.org/abs/1907.08610
    '''

    def __init__(self, optimizer, la_steps=5, la_alpha=0.8, use_locking=False, name="Lookahead"):
        """optimizer: inner optimizer
        la_steps (int): number of lookahead steps
        la_alpha (float): linear interpolation factor. 1.0 recovers the inner optimizer.
        """
        super(Lookahead, self).__init__(use_locking, name)
        self.optimizer = optimizer
        self._la_step = 0
        self._la_alpha = la_alpha
        self._total_la_steps = la_steps

    def _create_slots(self, var_list):
        self.optimizer._create_slots(var_list)

        self._var_list = var_list
        first_var = min(var_list, key=lambda x: x.name)
        self._create_non_slot_variable(initial_value=self._la_step,
                                       name="la_step",
                                       colocate_with=first_var)

        # Create slots for the cached parameters.
        for v in var_list:
            self._zeros_slot(v, "cached_params", self._name)

    def _prepare(self):
        self.optimizer._prepare()

        la_alpha = self._call_if_callable(self._la_alpha)
        total_la_steps = self._call_if_callable(self._total_la_steps)

        self._la_alpha_t = ops.convert_to_tensor(la_alpha, name="la_alpha")
        self._total_la_steps_t = ops.convert_to_tensor(total_la_steps, name="total_la_steps")

    def _get_la_step_accumulators(self):
        with ops.init_scope():
            if context.executing_eagerly():
                graph = None
            else:
                graph = ops.get_default_graph()
            return self._get_non_slot_variable("la_step", graph=graph)

    def _apply_dense(self, grad, var):
        return self.optimizer._apply_dense(grad, var)

    def _resource_apply_dense(self, grad, var):
        return self.optimizer._resource_apply_dense(grad, var)

    def _apply_sparse_shared(self, grad, var, indices, scatter_add):
        return self.optimizer._apply_sparse_shared(grad, var, indices, scatter_add)

    def _apply_sparse(self, grad, var):
        return self.optimizer._apply_sparse(grad, var)

    def _resource_scatter_add(self, x, i, v):
        return self.optimizer._resource_scatter_add(x, i, v)

    def _resource_apply_sparse(self, grad, var, indices):
        return self.optimizer._resource_apply_sparse(grad, var, indices)

    def _finish(self, update_ops, name_scope):
        inner_finish_op = self.optimizer._finish(update_ops, name_scope)

        with ops.control_dependencies([inner_finish_op, ]):
            la_step = self._get_la_step_accumulators()
            with ops.colocate_with(la_step):
                def update_la_step_func():
                    # update the la_step
                    return control_flow_ops.group([la_step.assign(
                        la_step + 1, use_locking=self._use_locking), ])

                def pull_back_func():
                    # update the la_step
                    update_la_step = la_step.assign(
                        0, use_locking=self._use_locking)
                    # interpolate the variables
                    interpolation = [v.assign(
                        self.get_slot(v, "cached_params") + self._la_alpha_t * (v - self.get_slot(v, "cached_params")))
                                     for v in self._var_list]

                    # update the cached params
                    with ops.control_dependencies(interpolation):
                        update_cached_params = [self.get_slot(v, "cached_params").assign(updated_v) for v, updated_v in
                                                zip(self._var_list, interpolation)]
                    return control_flow_ops.group([update_la_step, ] + interpolation + update_cached_params)

                # condition for when to pull back the params
                condition = tf.greater_equal(la_step, self._total_la_steps_t)
                update_lookahead_states = tf.cond(condition,
                                                  pull_back_func,
                                                  update_la_step_func,
                                                  )

        return control_flow_ops.group([inner_finish_op, update_lookahead_states],
                                      name=name_scope)

    def _call_if_callable(self, param):
        """Call the function if param is callable."""
        return param() if callable(param) else param

In [3]:
import numpy as np
(x_train, y_train), (x_test, y_test) = tf.keras.datasets.fashion_mnist.load_data()
x_train = np.expand_dims(x_train, -1)
x_test = np.expand_dims(x_test, -1)

4423680/4422102 [==============================] - 0s 0us/step


In [4]:
print(x_train.shape)
print(y_train.shape)

(60000, 28, 28, 1)
(60000,)


In [0]:
def create_model():
  model = tf.keras.models.Sequential()
  model.add(tf.keras.layers.Conv2D(64, (5, 5), padding='same', activation='elu'))
  model.add(tf.keras.layers.MaxPooling2D(pool_size=(2, 2), strides=(2,2)))

  model.add(tf.keras.layers.Conv2D(128, (5, 5), padding='same', activation='elu'))
  model.add(tf.keras.layers.MaxPooling2D(pool_size=(2, 2)))

  model.add(tf.keras.layers.Conv2D(256, (5, 5), padding='same', activation='elu'))
  model.add(tf.keras.layers.MaxPooling2D(pool_size=(2, 2), strides=(2,2)))

  model.add(tf.keras.layers.Flatten())
  model.add(tf.keras.layers.Dense(256))
  model.add(tf.keras.layers.Activation('elu'))
  model.add(tf.keras.layers.Dense(10))
  model.add(tf.keras.layers.Activation('softmax'))
  return model

In [0]:
model1 = create_model()

optimizer = tf.compat.v1.train.AdamOptimizer(1e-3)

model1.compile(
    optimizer=optimizer,
    loss='sparse_categorical_crossentropy',
    metrics=['sparse_categorical_accuracy']
)

In [0]:
model2 = create_model()

optimizer = tf.compat.v1.train.AdamOptimizer(1e-3)

model2.compile(
    optimizer=Lookahead(optimizer, la_steps=5, la_alpha=0.8),
    loss='sparse_categorical_crossentropy',
    metrics=['sparse_categorical_accuracy']
)

In [0]:
model3 = create_model()

optimizer = tf.compat.v1.train.AdamOptimizer(1e-3)

model3.compile(
    optimizer=Lookahead(optimizer, la_steps=10, la_alpha=0.8),
    loss='sparse_categorical_crossentropy',
    metrics=['sparse_categorical_accuracy']
)

In [0]:
model4 = create_model()

optimizer = tf.compat.v1.train.AdamOptimizer(1e-3)

model4.compile(
    optimizer=Lookahead(optimizer, la_steps=20, la_alpha=0.8),
    loss='sparse_categorical_crossentropy',
    metrics=['sparse_categorical_accuracy']
)

In [0]:
model5 = create_model()

optimizer = tf.compat.v1.train.AdamOptimizer(1e-3)

model5.compile(
    optimizer=Lookahead(optimizer, la_steps=5, la_alpha=0.6),
    loss='sparse_categorical_crossentropy',
    metrics=['sparse_categorical_accuracy']
)

In [0]:
model6 = create_model()

optimizer = tf.compat.v1.train.AdamOptimizer(1e-3)

model6.compile(
    optimizer=Lookahead(optimizer, la_steps=10, la_alpha=0.6),
    loss='sparse_categorical_crossentropy',
    metrics=['sparse_categorical_accuracy']
)

In [12]:
history1 = model1.fit(
    x_train.astype(np.float32), y_train.astype(np.float32),
    epochs=10,
    steps_per_epoch=600,
    validation_data=(x_test.astype(np.float32), y_test.astype(np.float32)),
    validation_freq=17
)

print(history1.history)

Epoch 1/10
600/600 [==============================] - 18s 30ms/step - loss: 1.0905 - sparse_categorical_accuracy: 0.8216
Epoch 2/10
600/600 [==============================] - 18s 30ms/step - loss: 0.2837 - sparse_categorical_accuracy: 0.8956
Epoch 3/10
600/600 [==============================] - 18s 30ms/step - loss: 0.2431 - sparse_categorical_accuracy: 0.9110
Epoch 4/10
600/600 [==============================] - 18s 30ms/step - loss: 0.2138 - sparse_categorical_accuracy: 0.9203
Epoch 5/10
600/600 [==============================] - 18s 30ms/step - loss: 0.1957 - sparse_categorical_accuracy: 0.9274
Epoch 6/10
600/600 [==============================] - 18s 30ms/step - loss: 0.1860 - sparse_categorical_accuracy: 0.9309
Epoch 7/10
600/600 [==============================] - 18s 30ms/step - loss: 0.1723 - sparse_categorical_accuracy: 0.9353
Epoch 8/10
600/600 [==============================] - 18s 30ms/step - loss: 0.1654 - sparse_categorical_accuracy: 0.9373
Epoch 9/10
600/600 [============

In [13]:
history2 = model2.fit(
    x_train.astype(np.float32), y_train.astype(np.float32),
    epochs=10,
    steps_per_epoch=600,
    validation_data=(x_test.astype(np.float32), y_test.astype(np.float32)),
    validation_freq=17
)

print(history2.history)

Epoch 1/10
600/600 [==============================] - 19s 31ms/step - loss: 0.8969 - sparse_categorical_accuracy: 0.8300
Epoch 2/10
600/600 [==============================] - 19s 31ms/step - loss: 0.2631 - sparse_categorical_accuracy: 0.9029
Epoch 3/10
600/600 [==============================] - 19s 31ms/step - loss: 0.2265 - sparse_categorical_accuracy: 0.9158
Epoch 4/10
600/600 [==============================] - 19s 31ms/step - loss: 0.1967 - sparse_categorical_accuracy: 0.9264
Epoch 5/10
600/600 [==============================] - 19s 31ms/step - loss: 0.1787 - sparse_categorical_accuracy: 0.9329
Epoch 6/10
600/600 [==============================] - 19s 31ms/step - loss: 0.1626 - sparse_categorical_accuracy: 0.9397
Epoch 7/10
600/600 [==============================] - 19s 31ms/step - loss: 0.1554 - sparse_categorical_accuracy: 0.9412
Epoch 8/10
600/600 [==============================] - 19s 31ms/step - loss: 0.1424 - sparse_categorical_accuracy: 0.9459
Epoch 9/10
600/600 [============

In [14]:
history3 = model3.fit(
    x_train.astype(np.float32), y_train.astype(np.float32),
    epochs=10,
    steps_per_epoch=600,
    validation_data=(x_test.astype(np.float32), y_test.astype(np.float32)),
    validation_freq=17
)

print(history3.history)

Epoch 1/10
600/600 [==============================] - 19s 31ms/step - loss: 0.9582 - sparse_categorical_accuracy: 0.8274
Epoch 2/10
600/600 [==============================] - 19s 31ms/step - loss: 0.2710 - sparse_categorical_accuracy: 0.9005
Epoch 3/10
600/600 [==============================] - 19s 31ms/step - loss: 0.2285 - sparse_categorical_accuracy: 0.9139
Epoch 4/10
600/600 [==============================] - 19s 31ms/step - loss: 0.1993 - sparse_categorical_accuracy: 0.9250
Epoch 5/10
600/600 [==============================] - 19s 31ms/step - loss: 0.1825 - sparse_categorical_accuracy: 0.9319
Epoch 6/10
600/600 [==============================] - 19s 31ms/step - loss: 0.1678 - sparse_categorical_accuracy: 0.9375
Epoch 7/10
600/600 [==============================] - 19s 31ms/step - loss: 0.1513 - sparse_categorical_accuracy: 0.9435
Epoch 8/10
600/600 [==============================] - 19s 31ms/step - loss: 0.1452 - sparse_categorical_accuracy: 0.9459
Epoch 9/10
600/600 [============

In [15]:
history4 = model4.fit(
    x_train.astype(np.float32), y_train.astype(np.float32),
    epochs=10,
    steps_per_epoch=600,
    validation_data=(x_test.astype(np.float32), y_test.astype(np.float32)),
    validation_freq=17
)

print(history4.history)

Epoch 1/10
600/600 [==============================] - 19s 31ms/step - loss: 1.0630 - sparse_categorical_accuracy: 0.8173
Epoch 2/10
600/600 [==============================] - 19s 31ms/step - loss: 0.2782 - sparse_categorical_accuracy: 0.8956
Epoch 3/10
600/600 [==============================] - 19s 31ms/step - loss: 0.2357 - sparse_categorical_accuracy: 0.9120
Epoch 4/10
600/600 [==============================] - 19s 31ms/step - loss: 0.2100 - sparse_categorical_accuracy: 0.9204
Epoch 5/10
600/600 [==============================] - 19s 31ms/step - loss: 0.1901 - sparse_categorical_accuracy: 0.9288
Epoch 6/10
600/600 [==============================] - 19s 31ms/step - loss: 0.1704 - sparse_categorical_accuracy: 0.9361
Epoch 7/10
600/600 [==============================] - 19s 31ms/step - loss: 0.1604 - sparse_categorical_accuracy: 0.9398
Epoch 8/10
600/600 [==============================] - 19s 31ms/step - loss: 0.1515 - sparse_categorical_accuracy: 0.9437
Epoch 9/10
600/600 [============

In [16]:
history5 = model5.fit(
    x_train.astype(np.float32), y_train.astype(np.float32),
    epochs=10,
    steps_per_epoch=600,
    validation_data=(x_test.astype(np.float32), y_test.astype(np.float32)),
    validation_freq=17
)

print(history5.history)

Epoch 1/10
600/600 [==============================] - 19s 31ms/step - loss: 0.8717 - sparse_categorical_accuracy: 0.8214
Epoch 2/10
600/600 [==============================] - 19s 31ms/step - loss: 0.2682 - sparse_categorical_accuracy: 0.9007
Epoch 3/10
600/600 [==============================] - 19s 31ms/step - loss: 0.2252 - sparse_categorical_accuracy: 0.9166
Epoch 4/10
600/600 [==============================] - 19s 31ms/step - loss: 0.1933 - sparse_categorical_accuracy: 0.9286
Epoch 5/10
600/600 [==============================] - 19s 31ms/step - loss: 0.1705 - sparse_categorical_accuracy: 0.9355
Epoch 6/10
600/600 [==============================] - 19s 31ms/step - loss: 0.1520 - sparse_categorical_accuracy: 0.9430
Epoch 7/10
600/600 [==============================] - 19s 31ms/step - loss: 0.1350 - sparse_categorical_accuracy: 0.9491
Epoch 8/10
600/600 [==============================] - 19s 31ms/step - loss: 0.1281 - sparse_categorical_accuracy: 0.9517
Epoch 9/10
600/600 [============

In [17]:
history6 = model6.fit(
    x_train.astype(np.float32), y_train.astype(np.float32),
    epochs=10,
    steps_per_epoch=600,
    validation_data=(x_test.astype(np.float32), y_test.astype(np.float32)),
    validation_freq=17
)

print(history6.history)

Epoch 1/10
600/600 [==============================] - 19s 31ms/step - loss: 1.0656 - sparse_categorical_accuracy: 0.8129
Epoch 2/10
600/600 [==============================] - 19s 31ms/step - loss: 0.2738 - sparse_categorical_accuracy: 0.8990
Epoch 3/10
600/600 [==============================] - 19s 31ms/step - loss: 0.2245 - sparse_categorical_accuracy: 0.9158
Epoch 4/10
600/600 [==============================] - 19s 31ms/step - loss: 0.1932 - sparse_categorical_accuracy: 0.9284
Epoch 5/10
600/600 [==============================] - 19s 31ms/step - loss: 0.1714 - sparse_categorical_accuracy: 0.9355
Epoch 6/10
600/600 [==============================] - 19s 31ms/step - loss: 0.1478 - sparse_categorical_accuracy: 0.9431
Epoch 7/10
600/600 [==============================] - 19s 31ms/step - loss: 0.1330 - sparse_categorical_accuracy: 0.9484
Epoch 8/10
600/600 [==============================] - 19s 31ms/step - loss: 0.1242 - sparse_categorical_accuracy: 0.9528
Epoch 9/10
600/600 [============

In [0]:
from google.colab import files
files.download('final_ans_2.txt') 


In [0]:
with open("final_ans_2.txt","w+") as fi:
  fi.write(', '.join([str(num) for num in history1.history["loss"]]))
  fi.write('\n')
  fi.write(', '.join([str(num) for num in history2.history["loss"]]))
  fi.write('\n')
  fi.write(', '.join([str(num) for num in history3.history["loss"]]))
  fi.write('\n')
  fi.write(', '.join([str(num) for num in history4.history["loss"]]))
  fi.write('\n')
  fi.write(', '.join([str(num) for num in history5.history["loss"]]))
  fi.write('\n')
  fi.write(', '.join([str(num) for num in history6.history["loss"]]))
  fi.write('\n')